In [ ]:
# CS 286 Project Spring 2021 under Professor  
#
# David Bui and Sushant Mane
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import copy
import pickle
import random
from os import listdir, walk
from os.path import isfile, join
import io
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

In [ ]:
# To run must go through folder and uncomment certain blocks of code
# basePath in drive should be changed according to how folders are placed 
basePath = '/content/drive/MyDrive/286Project/'

# sequence fasta file folders
no_react_folder = basePath + 'fastaNoReaction'
avicel_folder = basePath + 'fastaAvicel'
cmc_folder = basePath + 'fastaCMC'
lich_folder = basePath + 'fastaLichenan'
miscanthus_folder = basePath + 'fastaMiscanthus'
switchgrass_folder = basePath 'fastaSwitchgrass'
xylan_folder = basePath + 'fastaXylan'

# folders containing sequences that did not react to the substrate in name
no_react_not_folder = basePath + '/content/drive/MyDrive/286Project/fastaNoReactionNot'
avicel_not_folder = basePath + 'fastaAvicelNot'
cmc_not_folder = basePath + 'fastaCMCNot'
lich_not_folder = basePath + 'fastaLichenanNot'
miscanthus_not_folder = basePath + 'fastaMiscanthusNot'
switchgrass_not_folder = basePath + 'fastaSwitchgrassNot'
xylan_not_folder = basePath + 'fastaXylanNot'

savePath = basePath + 'mlp_cmc_model.sav' # path to save mlp models


# uncomment one pair below train whichever model or substrate focus
folders_to_classify = []

# folders_to_classify.append(no_react_folder)
# folders_to_classify.append(no_react_not_folder)

# folders_to_classify.append(avicel_folder)
# folders_to_classify.append(avicel_not_folder)

# folders_to_classify.append(cmc_folder)
# folders_to_classify.append(cmc_not_folder)

# folders_to_classify.append(lich_folder)
# folders_to_classify.append(lich_not_folder)

# folders_to_classify.append(miscanthus_folder)
# folders_to_classify.append(miscanthus_not_folder)

# folders_to_classify.append(switchgrass_folder)
# folders_to_classify.append(switchgrass_not_folder)

# folders_to_classify.append(xylan_folder)
# folders_to_classify.append(xylan_not_folder)


# functions below are for parsing and adding the sequences in each fasta file
# to a pandas dataframe

#
# Parse files and add to a id# -> sequence dictionary IDToSequence
#
def get_dir_files(dir): 
  files = [f for f in listdir(dir) if isfile(join(dir, f))]
  return files

def seq_map(folder, files): 
  IDToSequence = {}
  for next in files:
    nextPath = folder + '/'+ next
    sequenceName = next[:len(next)-4] # remove the .txt extension
    text_file = open(nextPath, "r")
    lines = text_file.readlines()
    sliced = lines[1:]
    combined = ''.join(sliced)
    IDToSequence[sequenceName] = combined.replace('\n', '').replace('\r', '')
  return IDToSequence

def seq_arr(folder, files):
  seq = []
  for next in files:
    nextPath = folder + '/'+ next
    sequenceName = next[:len(next)-4] # remove the .txt extension
    text_file = open(nextPath, "r")
    lines = text_file.readlines()
    sliced = lines[1:]
    combined = ''.join(sliced)
    seq.append(combined.replace('\n', '').replace('\r', ''))
  return seq


def print_seq(seq):
  for next in seq:
    name = 'Sequence ID: ' + str(next)
    sequence = 'Sequence below: \n' + seq[next]  + '\n'
    print(name)
    print(sequence)

def get_seq(data_folders = []):
  data_files = []
  for next in data_folders:
    data_files.append(get_dir_files(next))
  fasta_seq = []
  f_len = len(data_folders)
  for i in range(f_len):
    fasta_seq.append(seq_arr(data_folders[i], data_files[i]))
  return fasta_seq

# gather sequences
def addToData(lists): 
  classes = 0
  data = {'sequence': [], 'class': []}
  for seq in lists:
    prev = data['sequence']
    conca = prev + seq 
    seq_len = len(seq)
    data.update({'sequence': conca})
    class_list = []
    for i in range(seq_len):
      class_list.append(classes)
    prev_class = data['class']
    both_class = prev_class + class_list
    data.update({'class': both_class})
    classes = classes + 1
  return data


lists_of_seq = get_seq(folders_to_classify)
data = addToData(lists_of_seq)
cazy_data = pd.DataFrame(data)
cazy_data.head()

,sequence,class
0,ATGAAAAGAATAACCTCCATTGCAATCACTGCATTAATCTTCCTCA...,0
1,AACGGATTAGGGTTAAAAGGAGTACAAAAGATGAAAAAGACTCACG...,0
2,ATGAAAGCATGTATTTTTAGTATCGCCGCACTGATGCTTGCACTTA...,0
3,ATGAATAAAGAAAGATTGCGCGGAGTGAATTTGGGAGGATGGTTCA...,0
4,AGAAACATGAAAATAAGACATTTATTAATAATGGTATTTGCATGGT...,0


In [ ]:
# begin code for worksheet 6 baysian
# function to convert sequence strings into k-mer words, default size = 6 (hexamer words)
def getKmers(sequence, size=6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]
cazy_data['words'] = cazy_data.apply(lambda x: getKmers(x['sequence']), axis=1)
cazy_data = cazy_data.drop('sequence', axis=1)

cazy_texts = list(cazy_data['words'])
for item in range(len(cazy_texts)):
    cazy_texts[item] = ' '.join(cazy_texts[item])
y_data = cazy_data.iloc[:, 0].values    

y_data

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [ ]:
# Creating the Bag of Words model using CountVectorizer()
# This is equivalent to k-mer counting
# The n-gram size of 4 was previously determined by testing
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(4,4))
X_cazy = cv.fit_transform(cazy_texts)
print(X_cazy.shape)

(46, 53833)


In [ ]:
# Splitting the human dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_cazy, 
                                                    y_data, 
                                                    test_size = 0.25, random_state = 42) 

print("TRAINING X train and y train shape")
print(X_train.shape)
print(y_train.shape)
print("TESTING X test and y test shape")
print(X_test.shape)
print(y_test.shape)

TRAINING X train and y train
(34, 53833)
(34,)
TESTING X test and y test
(12, 53833)
(12,)


In [ ]:
# various tested ML models
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import ParameterGrid
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score

# Best mlp models found through grid search uncomment depending on 

# # no reaction enzymes and enzymes with any reactions: 1.00 accuracy in grid search
# classifier = MLPClassifier(activation='relu', hidden_layer_sizes=(10,10), learning_rate='adaptive', max_iter=1000000, solver='lbfgs', shuffle=False)

# # enzymes with avicel versus no avicel reactions 0.917: accuracy in grid search
# classifier = MLPClassifier(activation='relu', hidden_layer_sizes=(10,10,10), learning_rate='constant', max_iter=1000000, solver='lbfgs', shuffle=True)

# # enzymes with cmc versus no cmc reactions: 0.8333 accuracy in grid search
# classifier = MLPClassifier(activation='relu', hidden_layer_sizes=(20,20,20), learning_rate='adaptive', max_iter=1000000, solver='lbfgs', shuffle=True)

# # enzymes with lich versus not lich reactions: 1.000 accuracy in grid search
# classifier = MLPClassifier(activation='relu', hidden_layer_sizes=(10,10), learning_rate='constant', max_iter=1000000, solver='lbfgs', shuffle=True)

# # enzymes with miscanthus versus no miscanthus reactions 0.857 accuracy in grid search
# classifier = MLPClassifier(activation='relu', hidden_layer_sizes=(10,10), learning_rate='constant', max_iter=1000000, solver='lbfgs', shuffle=True)

# # enzymes with switchgrass versus  no switchgrass reactions:  0.857 accuracy in grid search
# classifier = MLPClassifier(activation='relu', hidden_layer_sizes=(10,10), learning_rate='constant', max_iter=1000000, solver='lbfgs', shuffle=True)

# # enzymes with xylan versus no xylan reactions: 1.000 accuracy in grid search
# classifier = MLPClassifier(activation='relu', hidden_layer_sizes=(10,10), learning_rate='constant', max_iter=1000000, solver='sgd', shuffle=False)

mlp = MLPClassifier()

mlp_params = {
'learning_rate': ["constant", "adaptive", "invscaling"],
'hidden_layer_sizes': [(10,10,),(20,20,),(30,30,),(10,10,10),(20,20,20), (30,30,30),(50,50),],
'solver': ['lbfgs','sgd', 'adam'],
'activation': ["relu", "logistic", "tanh"],
'max_iter' : [1000],
'shuffle': [False, True]
}


gridMLP = GridSearchCV(estimator=mlp, param_grid=mlp_params, scoring='accuracy', verbose=10)


# Uncomment below to run a MLP grid search on the current dataset. 
# Gridsearch accuracy is based on Training dataset only.
# WARNING This will take 1 - 2 to run uunder current parameter grid.


# gridMLP.fit(X_train, y_train)
# print('Best mlp')
# print('Best parameters found:\n', gridMLP.best_params_)
# print("Best score: ",gridMLP.best_score_)

classifier.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 20, 20), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
# The code below loops until a MLP classifier is found with > 90 % accuracy on the testing dataset.

y_pred = classifier.predict(X_test)
print(y_test)
print(y_pred)
print("Confusion matrix\n")
print(pd.crosstab(pd.Series(y_test, name='Actual'), pd.Series(y_pred, name='Predicted')))
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(y_test, y_pred)
print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))

# loop here to do random restarts
while accuracy < 0.91 or accuracy == 1.00:
  X_train, X_test, y_train, y_test = train_test_split(X_cazy, 
                                                    y_data, 
                                                    test_size = 0.20) #random_state = 42
  classifier.fit(X_train, y_train)
  y_pred = classifier.predict(X_test)
  accuracy, precision, recall, f1 = get_metrics(y_test, y_pred)
  print("Confusion matrix\n")
  print(pd.crosstab(pd.Series(y_test, name='Actual'), pd.Series(y_pred, name='Predicted')))
  print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))
  if (accuracy >= 0.900 and precision >= 0.910) and not (accuracy == 1.00 or precision == 1.00):
    break


[0 1 0 0 0 1 0 1 0 1 1 1]
[0 0 1 0 0 0 0 0 0 0 0 0]
Confusion matrix

Predicted  0  1
Actual         
0          5  1
1          6  0
accuracy = 0.417 
precision = 0.227 
recall = 0.417 
f1 = 0.294
Confusion matrix

Predicted  0  1
Actual         
0          6  1
1          2  1
accuracy = 0.700 
precision = 0.675 
recall = 0.700 
f1 = 0.680
Confusion matrix

Predicted  0  1
Actual         
0          3  1
1          3  3
accuracy = 0.600 
precision = 0.650 
recall = 0.600 
f1 = 0.600
Confusion matrix

Predicted  0  1
Actual         
0          2  7
1          0  1
accuracy = 0.300 
precision = 0.912 
recall = 0.300 
f1 = 0.349
Confusion matrix

Predicted  0  1
Actual         
0          7  1
1          2  0
accuracy = 0.700 
precision = 0.622 
recall = 0.700 
f1 = 0.659


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion matrix

Predicted  0
Actual      
0          7
1          3
accuracy = 0.700 
precision = 0.490 
recall = 0.700 
f1 = 0.576
Confusion matrix

Predicted  0  1
Actual         
0          4  2
1          2  2
accuracy = 0.600 
precision = 0.600 
recall = 0.600 
f1 = 0.600
Confusion matrix

Predicted  0  1
Actual         
0          7  0
1          2  1
accuracy = 0.800 
precision = 0.844 
recall = 0.800 
f1 = 0.763


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion matrix

Predicted  0
Actual      
0          5
1          5
accuracy = 0.500 
precision = 0.250 
recall = 0.500 
f1 = 0.333


...

Confusion matrix

Predicted  0
Actual      
0          7
1          3
accuracy = 0.700 
precision = 0.490 
recall = 0.700 
f1 = 0.576


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion matrix

Predicted  0
Actual      
0          8
1          2
accuracy = 0.800 
precision = 0.640 
recall = 0.800 
f1 = 0.711
Confusion matrix

Predicted  0  1
Actual         
0          6  3
1          1  0
accuracy = 0.600 
precision = 0.771 
recall = 0.600 
f1 = 0.675


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion matrix

Predicted  1
Actual      
0          6
1          4
accuracy = 0.400 
precision = 0.160 
recall = 0.400 
f1 = 0.229
Confusion matrix

Predicted  0  1
Actual         
0          7  0
1          1  2
accuracy = 0.900 
precision = 0.912 
recall = 0.900 
f1 = 0.893


In [ ]:
print("\n break \n")
print("Confusion matrix\n")
print(pd.crosstab(pd.Series(y_test, name='Actual'), pd.Series(y_pred, name='Predicted')))
print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))
pickle.dump(classifier, open(savePath, 'wb'))


 break 

Confusion matrix

Predicted  0  1
Actual         
0          7  0
1          1  2
accuracy = 0.900 
precision = 0.912 
recall = 0.900 
f1 = 0.893
